# Marty Stouffer Wildlife Art Gallery

Sales and Marketing Analysis (Q1 2021)\
(Actual sales data used; names have been changed to protect confidentiality)

In [ ]:
# Dependencies and setup
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import chardet
import gmaps

In [ ]:
# This is where the sales and orders data .CSV lives locally...:
ordersDataFilepath = "Data/heatmap_orders.csv"

print(f"{ordersDataFilepath} is {round(os.path.getsize(ordersDataFilepath)/1024/1024, 2)} megabytes (MB).")

In [ ]:
# Read CSV into orderData DataFrame
ordersData_df = pd.read_csv(ordersDataFilepath, encoding = "iso-8859-1")
ordersData_df

In [ ]:
# This is where the city lat/lng data .CSV lives locally...:
citiesDataFilepath = "Data/heatmap_cities.csv"

print(f"{citiesDataFilepath} is {round(os.path.getsize(citiesDataFilepath)/1024/1024, 2)} megabytes (MB).")

In [ ]:
# Read CSV into citiesData DataFrame
citiesData_df = pd.read_csv(citiesDataFilepath, encoding = "iso-8859-1")
citiesData_df

In [ ]:
# Merge ordersData and citiesData DataFrames
mergedData_df = pd.merge(ordersData_df, citiesData_df, how = "outer", on = ["city", "state_id"], sort = False, copy = False, indicator = False, validate = None)
mergedData_df

In [ ]:
# Remove city rows with no sales
mergedData_df = mergedData_df[mergedData_df["Subtotal_x"].notna()]
mergedData_df

In [ ]:
# Drop irrelevant columns
mergedData_df.drop(columns = ["Subtotal_y", "city_ascii"], inplace = True)
mergedData_df

In [ ]:
# Rename columns to clean up DataFrame
mergedData_df = mergedData_df.rename(columns = {"Subtotal_x":"Subtotal", "city":"City", "state_id":"State", "lat":"Latitude",
                                                "lng":"Longitude"})
mergedData_df

In [ ]:
# Export to CSV
mergedData_df.to_csv("ordersData_byLocation.csv", index = False, header = True)

# Gross Sales Revenues by State
(Q1 2021)

### Key Points:
1. Cleaned orders CSV removed 45 orders from Australia/Canada/Fiji/Great Britain totaling \\$9,831.12 (4.83\% of period sales).
2. Cleaned orders CSV removed 21 orders that were fully refunded totaling \\$3,610.69.
3. Result is \\$193,936.50 total period sales analyzed.

In [ ]:
# Groupby state
state_df = mergedData_df.groupby(["State"])

# Create variable to calculate subtotals by state
sales = state_df["Subtotal"].sum()

# Create new DataFrame to summarize sales by state
stateSummary_df = pd.DataFrame({"Subtotal":(sales)})
stateSummary_df.head()

In [ ]:
# Reset DataFrame index in place
stateSummary_df.reset_index(inplace = True)
stateSummary_df.head()

In [ ]:
# Create bar plot illustrating total sales by state

# Create variables for x and y values
state_x = stateSummary_df["State"]
state_y = stateSummary_df["Subtotal"]

# Define bar plot size
plt.figure(figsize = (18,10))

# Pass values into bar plot, and define color and transparency
plt.bar(state_x, state_y, color = "green", alpha = 0.43)

# Define bar plot title, x label, and y label (and their font sizes)
plt.title("Gross Sales Revenue by State", fontsize = 18)
plt.xlabel("State", fontsize = 14)
plt.ylabel("Total Sales ($)", fontsize = 14)

# Print pyplot bar plot to image file
plt.savefig("Images/Sales_by_State_Bar_Plot.jpg")

plt.show()

# Gross Sales Revenues by Geographic Area (HeatMap)
(Q1 2021)

In [ ]:
# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key = g_key)

In [ ]:
# Create heatmap for sales revenues by city location

# Store latitude and longitude in locations
locations = mergedData_df[["Latitude", "Longitude"]]

# Store sales revenues in weights
weights = mergedData_df["Subtotal"]

# Plot Heatmap (U.S. geographic center is 39.8333, -98.5855)
fig = gmaps.figure(zoom_level = 4.1, center = (37.8, -98.6))

# Set max intensity to highest sales revenue found in the dataset
max_intensity = mergedData_df["Subtotal"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = weights, 
                                 dissipating = False, max_intensity = max_intensity,
                                 point_radius = 1, gradient = ["white", "blue", "yellow", "red"])



# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Quartiles, Outliers, and Boxplots
(Q1 2021)

### Key Points:
1. Cleaned marketing CSV removed 21 orders that were fully refunded totaling \\$3,610.69.
2. Result is \\$203,412.13 total period sales analyzed.

In [ ]:
# This is where the marketing data .CSV lives locally...:
marketingDataFilepath = "Data/cleaned_marketing.csv"

print(f"{marketingDataFilepath} is {round(os.path.getsize(marketingDataFilepath)/1024/1024, 2)} megabytes (MB).")

In [ ]:
# Read CSV into marketingData DataFrame
marketingData_df = pd.read_csv(marketingDataFilepath, encoding = "iso-8859-1")
marketingData_df

In [ ]:
# Put answers into a list for for loop (and later for plot labels)
marketingList = ["No", "Yes"]

# Create empty list to fill with sales data (for plotting)
salesData = []

# Locate the rows which contain sales for each answer and get the sale amounts
for row in marketingList:
    sales_df = marketingData_df.loc[marketingData_df["Accepts Marketing"] == row]
    
    # Append list with sale amounts
    sale = sales_df["Subtotal"]
    salesData.append(sale)

    # Calculate the IQR and quartiles 
    quartiles = sale.quantile([0.25, 0.50, 0.75])
    lowerq = quartiles[0.25]
    medianq = quartiles[0.50]
    upperq = quartiles[0.75]
    iqr = upperq - lowerq

    # Print results to the display
    print(f"The lower quartile for those who answered {row} is:  {lowerq}.")
    print(f"The median value for those who answered {row} is:  {medianq}.")
    print(f"The upper quartile for those who answered {row} is:  {upperq}.")
    print(f"The interquartile range for those who answered {row} is:  {iqr}.")
    
    # Quantively determine outliers using upper and lower bounds and print to the display
    lower_bound = lowerq - (1.5 * iqr)
    upper_bound = upperq + (1.5 * iqr)
    print(f"Values below {lower_bound} could be outliers.")
    print(f"Values above {upper_bound} could be outliers.")
    print("---------------------------------------------------------------")

In [ ]:
# Generate box plots of the sale amounts across yes or no to marketing list
outlier = dict(marker = "o", markerfacecolor = "red", markersize = 6, markeredgecolor = "black")

# Define box plot size
plt.figure(figsize = (18,16))

# Define box plot values and outlier
plt.boxplot(salesData, labels = marketingList, flierprops = outlier)

# Define box plot title, y label, x label, and x tick labels
plt.title("Ranges of Sale Amounts per Marketing List", fontsize = 18)
plt.ylabel("Sale Amounts ($)", fontsize = 14)
plt.xlabel("On Marketing List?", fontsize = 14)
plt.xticks(fontsize = 14)

plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

# Print box plot to image file
plt.savefig("Images/Sale_Amounts_by_Marketing.jpg")

plt.show()

# Gross Sales Revenue vs. Marketing
(Q1 2021)

In [ ]:
# Do those on the marketing list result in higher sales?

# Retrieve marketing list and sales data
marketing = marketingData_df["Marketing Score"]
sales = marketingData_df["Subtotal"]
n = len(marketingData_df)

# Perform a linear regression on marketing list versus sales
slope, int, r, p, std_err = st.linregress(marketing, sales)

# Create equation of line to calculate predicted sales
fit = slope * marketing + int

# Create equation in string formats to print on scatter plot
equation = "y = " + str(round(slope, 2)) + "x + " + str(round(int, 2))

# Define scatter plot size
plt.figure(figsize = (18, 10))

# Plot x and y values on scatter plot
plt.scatter(marketing, sales, marker = ".", color = "black")

# Plot linear regression line on scatter plot
plt.plot(marketing, fit, "--", color = "red")

# Define linear regression line and print on scatter plot
plt.annotate(equation, (0.22, 350), fontsize = 14, color = "red")

# Define scatter plot title date, and x and y labels (and their font sizes)

plt.title(f"Gross Sales vs. Marketing List", fontsize = 18)
plt.xlabel("On Marketing List?  (0 = No, 1 = Yes)", fontsize = 14)
plt.ylabel("Sale Amounts ($)", fontsize = 14)
plt.grid(axis = "x", linewidth = 0.5)
plt.grid(axis = "y", linewidth = 0.5)

print(f"The r-value is: {r}")

# Print scatterplot to image file
# plt.savefig("Images/Sales_vs_Marketing_Scatterplot.png")

plt.show()

# Total Sales Revenue per Marketing List
(Q1 2021)

In [ ]:
# Group by marketing list and calculate total sales
total_sale = marketingData_df.groupby("Accepts Marketing").sum().round(2)
total_sale

In [ ]:
# Drop irrelevant columns
total_sale.drop(columns = ["Marketing Score"], inplace = True)
total_sale

In [ ]:
# Rename columns to clean up DataFrame
total_sale = total_sale.rename(columns = {"Subtotal":"Total Sales"})
total_sale

In [ ]:
# Declare variables to retrieve and store values for plotting
x_total = total_sale.index
y_total = total_sale["Total Sales"]

# Define colors for each bar
colors = ["red", "green"]

# Define bar plot size
plt.figure(figsize = (18,10))

# Pass values into bar plot, and define color and transparency
plt.bar(x_total, y_total, color = colors, alpha = 0.43)

# Define bar plot title, x label, and y label (and their font sizes)
plt.title("Total Sales Revenue per Marketing List", fontsize = 18)
plt.xlabel("On Marketing List?", fontsize = 14)
plt.ylabel("Total Sales ($)", fontsize = 14)
plt.ylim(0,200000)

# Print results to the display
print(f"Total Sales per {y_total}")

# Print pyplot bar plot to image file
plt.savefig("Images/Total_Sales_by_Marketing_Bar_Plot.jpg")

plt.show()

# Average Sale Revenue per Marketing List
(Q1 2021)

In [ ]:
# Group by marketing list and calculate average sale
average_sale = marketingData_df.groupby("Accepts Marketing").mean().round(2)
average_sale

In [ ]:
# Drop irrelevant columns
average_sale.drop(columns = ["Marketing Score"], inplace = True)
average_sale

In [ ]:
# Rename columns to clean up DataFrame
average_sale = average_sale.rename(columns = {"Subtotal":"Average Sale"})
average_sale

In [ ]:
# Declare variables to retrieve and store values for plotting
x_avg = average_sale.index
y_avg = average_sale["Average Sale"]

# Define colors for each bar
colors = ["red", "green"]

# Define bar plot size
plt.figure(figsize = (18,10))

# Pass values into bar plot, and define color and transparency
plt.bar(x_avg, y_avg, color = colors, alpha = 0.43)

# Define bar plot title, x label, and y label (and their font sizes)
plt.title("Average Sales Revenue per Marketing List", fontsize = 18)
plt.xlabel("On Marketing List?", fontsize = 14)
plt.ylabel("Average Sale ($)", fontsize = 14)
plt.ylim(0,300)

# Print results to the display
print(f"Average Sales per {y_avg}")

# Print pyplot bar plot to image file
plt.savefig("Images/Avg_Sales_by_Marketing_Bar_Plot.jpg")

plt.show()

# Conclusions
1. Increase Search Engine Optimization (SEO) for wildlife art searches in areas east of the Mississippi River.
2. Total sales are roughly 3X higher for those on the marketing list over those who are not (possibly due to large outliers).